In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("/content/Thyroid_Diff.csv")


In [4]:
data

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Thyroid Function,Physical Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response,Recurred
0,27,F,No,No,No,Euthyroid,Single nodular goiter-left,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Indeterminate,No
1,34,F,No,Yes,No,Euthyroid,Multinodular goiter,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
2,30,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
3,62,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
4,62,F,No,No,No,Euthyroid,Multinodular goiter,No,Micropapillary,Multi-Focal,Low,T1a,N0,M0,I,Excellent,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,72,M,Yes,Yes,Yes,Euthyroid,Single nodular goiter-right,Right,Papillary,Uni-Focal,High,T4b,N1b,M1,IVB,Biochemical Incomplete,Yes
379,81,M,Yes,No,Yes,Euthyroid,Multinodular goiter,Extensive,Papillary,Multi-Focal,High,T4b,N1b,M1,IVB,Structural Incomplete,Yes
380,72,M,Yes,Yes,No,Euthyroid,Multinodular goiter,Bilateral,Papillary,Multi-Focal,High,T4b,N1b,M1,IVB,Structural Incomplete,Yes
381,61,M,Yes,Yes,Yes,Clinical Hyperthyroidism,Multinodular goiter,Extensive,Hurthel cell,Multi-Focal,High,T4b,N1b,M0,IVA,Structural Incomplete,Yes


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [20]:
df = pd.read_csv("/content/Thyroid_Diff.csv")

In [21]:
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Smoking'] = label_encoder.fit_transform(df['Smoking'])
df['Hx Smoking'] = label_encoder.fit_transform(df['Hx Smoking'])
df['Hx Radiothreapy'] = label_encoder.fit_transform(df['Hx Radiothreapy'])
df['Thyroid Function'] = label_encoder.fit_transform(df['Thyroid Function'])
df['Physical Examination'] = label_encoder.fit_transform(df['Physical Examination'])
df['Adenopathy'] = label_encoder.fit_transform(df['Adenopathy'])
df['Pathology'] = label_encoder.fit_transform(df['Pathology'])
df['Focality'] = label_encoder.fit_transform(df['Focality'])
df['Risk'] = label_encoder.fit_transform(df['Risk'])
df['Stage'] = label_encoder.fit_transform(df['Stage'])

In [39]:
from sklearn.preprocessing import StandardScaler
df_subset_encoded = pd.get_dummies(df[['Age', 'Smoking', 'Risk', 'T', 'N', 'M', 'Stage', 'Response', 'Recurred']], drop_first=True)

X = df_subset_encoded.drop('Recurred_Yes', axis=1).values
y = df_subset_encoded['Recurred_Yes'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [41]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [44]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_resampled.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


history = model.fit(X_train_resampled, y_train_resampled, epochs=50, batch_size=8, validation_data=(X_test, y_test))

Epoch 1/50
55/55 [==============================] - 1s 6ms/step - loss: 0.5731 - accuracy: 0.7465 - val_loss: 0.2720 - val_accuracy: 0.9610
Epoch 2/50
55/55 [==============================] - 0s 3ms/step - loss: 0.2176 - accuracy: 0.9470 - val_loss: 0.1547 - val_accuracy: 0.9481
Epoch 3/50
55/55 [==============================] - 0s 2ms/step - loss: 0.1340 - accuracy: 0.9608 - val_loss: 0.1131 - val_accuracy: 0.9481
Epoch 4/50
55/55 [==============================] - 0s 2ms/step - loss: 0.1043 - accuracy: 0.9724 - val_loss: 0.1185 - val_accuracy: 0.9351
Epoch 5/50
55/55 [==============================] - 0s 2ms/step - loss: 0.0925 - accuracy: 0.9700 - val_loss: 0.1120 - val_accuracy: 0.9481
Epoch 6/50
55/55 [==============================] - 0s 2ms/step - loss: 0.0838 - accuracy: 0.9677 - val_loss: 0.1019 - val_accuracy: 0.9481
Epoch 7/50
55/55 [==============================] - 0s 2ms/step - loss: 0.0777 - accuracy: 0.9770 - val_loss: 0.1068 - val_accuracy: 0.9481
Epoch 8/50
55/55 [==

In [45]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

3/3 [==============================] - 0s 6ms/step - loss: 0.2436 - accuracy: 0.9221
Test Accuracy: 92.21%


In [46]:
import pickle

In [47]:
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)